# Data Evaluation <a id='data-evaluation'>

In [109]:
# standard libraries
import pandas as pd
import numpy as np
import os
from IPython.display import Image

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from tabulate import tabulate

# data pre-processing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# prediction models
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.impute import SimpleImputer

#from kneed import KneeLocator
from scipy import stats
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# import warnings filter
'''import warnings
warnings.filterwarnings('ignore')
from warnings import simplefilter 
simplefilter(action='ignore', category=FutureWarning)'''

"import warnings\nwarnings.filterwarnings('ignore')\nfrom warnings import simplefilter \nsimplefilter(action='ignore', category=FutureWarning)"

# Loading Data <a id='loading-data'>

In [2]:
url = 'https://raw.githubusercontent.com/olmosjorge28/QTW-SPRING-2022/main/dataset_diabetes/IDs_mapping.csv'
admission_type_mapping = pd.read_csv(url, nrows=8, index_col=0)
admission_type_mapping

,description
admission_type_id,
1,Emergency
2,Urgent
3,Elective
4,Newborn
5,Not Available
6,NaN
7,Trauma Center
8,Not Mapped


Can Potentially Deduce values from medical specialty

In [3]:
discharge_disposition_mapping = pd.read_csv(url, nrows=30,skiprows=10, index_col=0 )
discharge_disposition_mapping 

,description
discharge_disposition_id,
1,Discharged to home
2,Discharged/transferred to another short term h...
3,Discharged/transferred to SNF
4,Discharged/transferred to ICF
5,Discharged/transferred to another type of inpa...
6,Discharged/transferred to home with home healt...
7,Left AMA
8,Discharged/transferred to home under care of H...
9,Admitted as an inpatient to this hospital


In [4]:
admission_source_mapping = pd.read_csv(url,skiprows=42, index_col=0 )
admission_source_mapping

,description
admission_source_id,
1,Physician Referral
2,Clinic Referral
3,HMO Referral
4,Transfer from a hospital
5,Transfer from a Skilled Nursing Facility (SNF)
6,Transfer from another health care facility
7,Emergency Room
8,Court/Law Enforcement
9,Not Available


# Referral,Transfer,Emergency, Normal Delivery, Other Delivery, Other/NA

In [5]:
url = 'https://raw.githubusercontent.com/olmosjorge28/QTW-SPRING-2022/main/dataset_diabetes/diabetic_data.csv'
df = pd.read_csv(url,na_values='?')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
df.columns.values.tolist()

['encounter_id',
 'patient_nbr',
 'race',
 'gender',
 'age',
 'weight',
 'admission_type_id',
 'discharge_disposition_id',
 'admission_source_id',
 'time_in_hospital',
 'payer_code',
 'medical_specialty',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'diag_1',
 'diag_2',
 'diag_3',
 'number_diagnoses',
 'max_glu_serum',
 'A1Cresult',
 'metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'acetohexamide',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'troglitazone',
 'tolazamide',
 'examide',
 'citoglipton',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'glimepiride-pioglitazone',
 'metformin-rosiglitazone',
 'metformin-pioglitazone',
 'change',
 'diabetesMed',
 'readmitted']

## Types of Data

### Qualitative

#### Nominal:

#### Ordinal:

### Quantitative

#### Discrete

#### Continous

## Remap Id Mappings

In [7]:
def remapIds(*new_mappings: tuple) -> dict:
    mapping_dict = dict()
    for mapping in new_mappings:
        mapping_dict[mapping[1]] =  mapping[0].map(mapping[2])
    return mapping_dict
        

In [8]:
def refactorMappingIds(inputDf: pd.DataFrame) -> pd.DataFrame:
    
    admission_type_id_new_mapping = {
        1: "emergency",
        2: "urgent",
        3: "elective",
        4: "newborn",
        5: float("NaN"),
        6: float("NaN"),
        7: "trauma-center",
        8: float("NaN")
    }
    
    disharge_disposition_new_mapping = {
        1: "discharged",
        2: "transfer",
        3: "transfer",
        4: "transfer",
        5: "transfer",
        6: "transfer",
        7: "ama",
        8: "transfer",
        9: "admitted",
        10: "transfer",
        11: "expired",
        12: "admitted",
        13: "hospice",
        14: "hospice",
        15: "transfer",
        16: "transfer",
        17: "transfer",
        18: float("NaN"),
        19: "expired",
        20: "expired",
        21: "expired",
        22: "transfer",
        23: "transfer",
        24: "transfer",
        25: float("NaN"),
        26: float("NaN"),
        27: "transfer",
        28: "transfer",
        29: "transfer",
    }


    admission_sourcing_new_mapping = {
        1: "referral",
        2: "referral",
        3: "referral",
        4: "transfer",
        5: "transfer",
        6: "transfer",
        7: "emergency",
        8: "law-enforcement",
        9: float("NaN"),
        10: "transfer",
        11: "normal-delivery",
        12: "other-delivery",
        13: "other-delivery",
        14: "other-delivery",
        15: float("NaN"),
        17: float("NaN"),
        18: "transfer",
        19: "transfer",
        20: float("NaN"),
        21: float("NaN"),
        22: "transfer",
        23: "normal-delivery",
        24: "normal-delivery",
        25: "transfer",
        26: "transfer"
    }
    df = inputDf.copy()
    mapping_tuples = [
        (df['discharge_disposition_id'],'disharge_disposition_new_mapping', disharge_disposition_new_mapping),
        (df['admission_source_id'],'admission_source_new_mapping', admission_sourcing_new_mapping),
        (df['admission_type_id'], 'admission_type_id_new_mapping', admission_type_id_new_mapping)
    ]
    remappings = remapIds(*mapping_tuples)
    for newMappingKey in remappings:
        df[newMappingKey] = remappings[newMappingKey]
    return df

## Recategorizing Diag_1, Diag_2, Diag_3

In [9]:
def getDiagCategory(input: float) -> str:
    val: str
    if input < 1:
        val = float("NaN")
    elif input < 140:
        val = 'A'
    elif input < 240:
        val = 'B'
    elif input < 280:
        val = 'C'
    elif input < 290:
        val = 'D'
    elif input < 320:
        val = 'E'
    elif input < 390:
        val = 'F'
    elif input < 460:
        val = 'G'
    elif input < 520:
        val = 'H'
    elif input < 580:
        val = 'I'
    elif input < 630:
        val = 'J'
    elif input < 680:
        val = 'K'
    elif input < 710:
        val = 'L'
    elif input < 740:
        val = 'M'
    elif input < 760:
        val = 'N'
    elif input < 780:
        val = 'O'
    elif input < 800:
        val = 'P'
    elif input < 1000:
        val = 'Q'
    elif input < 2000:
        val = 'R'
    elif input < 3000:
        val = 'S'
    else:
        val = 'Z'
    return val
    

def categorizeDiag(diag: pd.Series) -> pd.Series:
    df = diag.copy()
    df.fillna(0,inplace=True)
    df.mask(df.str.startswith('V', na=False),1000, inplace=True)
    df.mask(df.str.startswith('E', na=False),2000, inplace=True)
    df = pd.to_numeric(df)
    df = df.map(getDiagCategory)
    return df
    

In [10]:
def recategorizeDiags(inputDf: pd.DataFrame) -> pd.DataFrame:
    df = inputDf.copy()
    df['diag_1_categorized'] = categorizeDiag(df['diag_1'])
    df['diag_2_categorized'] = categorizeDiag(df['diag_2'])
    df['diag_3_categorized'] = categorizeDiag(df['diag_3'])
    return df
    

### Inserting New Variables Addition variables

In [11]:
def addDiag(*diags):
    df = None
    for diag in diags:
        if (df is None):
            df = diag.notna().astype(int)
        else:
            df = df + diag.notna().astype(int)
    return df

In [12]:
def recategorizeData(inputDf: pd.DataFrame) -> pd.DataFrame:
    df = inputDf.copy()
    df = refactorMappingIds(df)
    df = recategorizeDiags(df)
    df['total_diag'] = addDiag(df['diag_1'],df['diag_2'],df['diag_3'])
    return df
    

In [13]:
df_recategorized = recategorizeData(df)

## Missing Data

In [14]:
df.isna().sum()

encounter_id                    0
patient_nbr                     0
race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum                   0
A1Cresult                       0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

In [15]:
df_recategorized.isna().sum()

encounter_id                            0
patient_nbr                             0
race                                 2273
gender                                  0
age                                     0
weight                              98569
admission_type_id                       0
discharge_disposition_id                0
admission_source_id                     0
time_in_hospital                        0
payer_code                          40256
medical_specialty                   49949
num_lab_procedures                      0
num_procedures                          0
num_medications                         0
number_outpatient                       0
number_emergency                        0
number_inpatient                        0
diag_1                                 21
diag_2                                358
diag_3                               1423
number_diagnoses                        0
max_glu_serum                           0
A1Cresult                         

## Variables Not Needed

patient_nbr <br>
encounter_id <br>
examide <br>
citoglipton <br>
weight <br>

In [16]:
df_recategorized = df_recategorized.drop(['patient_nbr','encounter_id', 'examide', 'weight','citoglipton'], 1)

## Variables Removed Because of Recategorization

discharge_disposition_id
admission_source_id
admission_type_id
diag_1
diag_2
diag_3

In [17]:
df_recategorized = df_recategorized.drop(['discharge_disposition_id','admission_source_id','admission_type_id','diag_1',
                'diag_2','diag_3'], 1)

KNN, Total Elimnation, Mode/Mean/Median

https://www.verywellhealth.com/finding-icd-codes-2615311

https://www.aapc.com/codes/icd9-codes-range/

In [18]:
#for column in ['disharge_disposition_new_mapping', 'admission_source_new_mapping', 'admission_type_id_new_mapping', 'diag_1_categorized', 'diag_2_categorized', 'diag_3_categorized', 'race', 'payer_code']:
#     df2[column].fillna(df2[column].mode()[0], inplace=True)

In [19]:
#grouped_modes = df2.groupby(['admission_type_id_new_mapping'])['medical_specialty'].agg(pd.Series.mode)
#grouped_modes['trauma-center'] = grouped_modes['urgent']
#twoColumns = df2[['admission_type_id_new_mapping','medical_specialty']]


In [20]:
def fitImputers(x_train: pd.DataFrame): 
    df = x_train.copy()
    imputer = SimpleImputer(missing_values = np.nan,
                        strategy ='most_frequent')
    columns_of_interest = ['disharge_disposition_new_mapping', 'admission_source_new_mapping', 'admission_type_id_new_mapping', 'diag_1_categorized', 'diag_2_categorized', 'diag_3_categorized', 'race', 'payer_code']
    imputer.fit(df[columns_of_interest])
    
    grouped_modes = df.groupby(['admission_type_id_new_mapping'])['medical_specialty'].agg(pd.Series.mode)
    grouped_modes['trauma-center'] = grouped_modes['urgent']
    
    def customApply(input):
        if(pd.isnull(input[1])):
            input[1] = grouped_modes[input[0]]
        return input
    return imputer, customApply
    
    

In [100]:
def imputeData(x_train: pd.DataFrame, x_test: pd.DataFrame):
    imputed_train = x_train.copy()
    imputed_test = x_test.copy()

    imputer, customApply = fitImputers(x_train)
    columns_of_interest = ['disharge_disposition_new_mapping', 'admission_source_new_mapping', 'admission_type_id_new_mapping', 'diag_1_categorized', 'diag_2_categorized', 'diag_3_categorized', 'race', 'payer_code']
    imputed_train[columns_of_interest] = imputer.transform(imputed_train[columns_of_interest])
    imputed_test[columns_of_interest] = imputer.transform(imputed_test[columns_of_interest])
    
    twoColumnsTrain = imputed_train[['admission_type_id_new_mapping','medical_specialty']]
    twoColumnsTest = imputed_test[['admission_type_id_new_mapping','medical_specialty']]
    
    twoColumnsTrain.apply(customApply, axis=1);
    twoColumnsTest.apply(customApply, axis=1);
    
    
    imputed_train['medical_specialty'] = twoColumnsTrain['medical_specialty']
    imputed_test['medical_specialty'] = twoColumnsTest['medical_specialty']
    
    imputed_train.to_csv('diabetic_data_train_imputed.csv')
    imputed_train = pd.read_csv('diabetic_data_train_imputed.csv')
    
    
    imputed_test.to_csv('diabetic_data_test_imputed.csv')
    imputed_test = pd.read_csv('diabetic_data_test_imputed.csv')
    
    imputed_train = imputed_train.drop(['Unnamed: 0'], 1)
    imputed_test = imputed_test.drop(['Unnamed: 0'], 1)
    
    return imputed_train, imputed_test

## Model preparation

In [24]:
def reclassifyY(series: pd.Series, condition):
    s_copy = series.copy()
    s_copy = s_copy==condition
    s_copy = s_copy.astype(int)
    return s_copy

In [23]:
def prepareDataframe(df: pd.DataFrame, y_var: str, condition):
    X = df.copy()
    y = reclassifyY(X[y_var], condition)
    X = X.drop([y_var], axis=1)
    return X, y

In [25]:
X, y = prepareDataframe(df_recategorized, 'readmitted', '<30')

In [26]:
def scale_and_split(X, y, test_split, random):
    return scale_and_encode_data(*train_test_split(X, y, test_size = test_split, random_state= random)) 

In [83]:
def scale_and_encode_data(x_train, x_test, y_train, y_test):
    x_train, x_test = imputeData(x_train, x_test)
    
    scl = StandardScaler()
    cont_vars = x_train._get_numeric_data().columns
    scl.fit(x_train[cont_vars])
    X_train_scaled = pd.DataFrame( scl.transform(x_train[cont_vars]), columns = cont_vars, index = x_train.index) # apply to training
    X_test_scaled = pd.DataFrame( scl.transform(x_test[cont_vars]), columns = cont_vars, index = x_test.index ) # apply to the test set (without snooping)

    X_train_cat = x_train.drop(cont_vars, 1)
    X_test_cat = x_test.drop(cont_vars, 1)
    
    enc = OneHotEncoder(handle_unknown = 'ignore')
    enc.fit(X_train_cat)

    X_train_cat_e = pd.DataFrame( enc.transform(X_train_cat).toarray(), columns = enc.get_feature_names(), index = x_train.index)
    X_test_cat_e = pd.DataFrame(enc.transform(X_test_cat).toarray(), columns = enc.get_feature_names(), index = x_test.index)
    X_train_scaled = pd.concat([X_train_scaled, X_train_cat_e], axis=1)
    X_test_scaled = pd.concat([X_test_scaled, X_test_cat_e], axis=1)
    
    return X_train_scaled, X_test_scaled, y_train, y_test

,race,gender,age,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,metformin-pioglitazone,change,diabetesMed,disharge_disposition_new_mapping,admission_source_new_mapping,admission_type_id_new_mapping,diag_1_categorized,diag_2_categorized,diag_3_categorized,total_diag
0,AfricanAmerican,Female,[30-40),1,MC,InternalMedicine,34,0,9,0,...,No,No,No,discharged,emergency,emergency,G,G,C,3
1,Caucasian,Female,[80-90),5,HM,InternalMedicine,64,2,24,0,...,No,No,No,transfer,referral,emergency,G,G,G,3
2,Caucasian,Female,[60-70),3,MC,InternalMedicine,28,0,21,0,...,No,Ch,Yes,discharged,emergency,emergency,L,L,G,3
3,Caucasian,Male,[50-60),2,SP,InternalMedicine,46,0,17,0,...,No,Ch,Yes,discharged,emergency,emergency,G,L,G,3
4,AfricanAmerican,Male,[30-40),2,MC,InternalMedicine,39,0,6,0,...,No,No,Yes,discharged,emergency,emergency,C,C,G,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30525,Caucasian,Male,[70-80),4,MC,Surgery-Cardiovascular/Thoracic,58,6,54,0,...,No,Ch,Yes,discharged,emergency,emergency,G,G,C,3
30526,Caucasian,Male,[50-60),2,SP,Emergency/Trauma,8,3,9,0,...,No,Ch,Yes,transfer,emergency,urgent,G,G,G,3
30527,Caucasian,Male,[30-40),1,DM,Family/GeneralPractice,38,0,2,1,...,No,No,No,discharged,emergency,emergency,Q,S,C,3
30528,Caucasian,Female,[90-100),6,MC,Emergency/Trauma,54,0,17,0,...,No,Ch,Yes,transfer,emergency,urgent,H,C,C,3


In [86]:
X_train, X_test, y_train, y_test = scale_and_split(X,y,0.3,234523233)

(71236, 45)
(30530, 45)


In [119]:
def runLGR(penalty, c, cv_number, x_train, x_test, y_train, y_test):
    lgr = LogisticRegression(penalty=penalty, C=c, solver='liblinear', class_weight='balanced')
#     scoring = ['accuracy','precision','recall','f1','roc_auc']
#     lgr_cont = cross_validate(lgr, x_train, y_train, cv = 3, scoring=scoring)
    
    
#     print( 'cv accuracies:', lgr_cont['test_accuracy'] ) # accuracies from 3 folds
#     print( 'cv precisions:', lgr_cont['test_precision'] ) 
#     print( 'cv recalls:   ', lgr_cont['test_recall'] ) 
#     print( 'cv f1s:       ', lgr_cont['test_f1'] )
#     print( 'cv AUCs:      ', lgr_cont['test_roc_auc'] ) 
    
    lgr.fit(x_train,y_train)
    y_hat = lgr.predict(x_test)
    return y_hat, y_test
    

In [102]:
lgr, y_hat, y_test = runLGR('l2', 0.5, 3,*scale_and_split(X,y,0.3,234523233))

In [ ]:
cnf_matrix

In [106]:
confusion_matrix(y_test, y_hat)

array([[17599,  9484],
       [ 1510,  1937]])

In [108]:
classification_report(y_test, y_hat)

NameError: name 'classification_report' is not defined

In [120]:
y_hat, y_test = runLGR('l1', 0.5, 3,*scale_and_split(X,y,0.3,234523233))

In [122]:
confusion_matrix(y_test, y_hat)

array([[17628,  9455],
       [ 1514,  1933]])

In [123]:
print("Test Set\n",  classification_report(y_test, y_hat) )

Test Set
               precision    recall  f1-score   support

           0       0.92      0.65      0.76     27083
           1       0.17      0.56      0.26      3447

    accuracy                           0.64     30530
   macro avg       0.55      0.61      0.51     30530
weighted avg       0.84      0.64      0.71     30530

